In [382]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import sys
import csv
import os
from datetime import datetime
from sklearn import metrics
from dateutil.parser import parse
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline
import lightgbm as lgb
import numpy as np, pandas as pd, os
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import random
import string

In [394]:
import seaborn as sns
import math
import re
import warnings

import lightgbm as lgb
from unidecode import unidecode
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from itertools import combinations
from datetime import datetime

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
plt.style.use('ggplot')

In [339]:
data_version_input = "01-10-2020"

In [385]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

In [341]:

f =  "currentLocationName"
vc1 = sorted(df_train[f].value_counts().index)
print(vc1)
print(len(vc1))
print("#####################")
vc2 = sorted(df_test[f].value_counts().index)
print(vc2)
print(len(vc2))
a = []
for i in vc2:
    if i not in vc1:
        a.append(i)
print(len(a))
print(a)


['A Luoi (1), Binh Tri Thien, Vietnam', 'Abu Dhabi, United Arab Emirates', 'Ajaccio', 'American Falls, Idaho', 'Amherst, Massachusetts', 'An Biên', 'An Duong, Hải Phòng, Vietnam', 'An Giang (4), Nghia Binh, Vietnam', 'An Khê, Gia Lai, Vietnam', 'An Lac, Thái Bình, Vietnam', 'An Lão, Bình Ðịnh, Vietnam', 'An Lê, Thái Bình, Vietnam', 'An Nhơn', 'An Ninh, Long An, Vietnam', 'An Ninh, Quảng Ninh (Quảng Bình)', 'An Phong, Ðồng Tháp, Vietnam', 'An Phu', 'An Phú (thị trấn)', 'An Thuan, Song Be, Vietnam', 'Angiang, An Giang, Vietnam', 'Anh Son', 'Anh Sơn', 'Anini-y, Antique', 'Ap An Thai (1), Tiền Giang, Vietnam', 'Ap Bao Binh, Vietnam', 'Ap Ben Cam, Vietnam', 'Ap Ben Cau, Tây Ninh, Vietnam', 'Ap Binh Duong (1), Vietnam', 'Ap Binh Hung, Tiền Giang, Vietnam', 'Ap Binh Quang, Long An, Vietnam', 'Ap Binh Thuan, Minh Hai, Vietnam', 'Ap Hung Yen, Song Be, Vietnam', 'Ap Lam Phu, Lâm Ðồng, Vietnam', 'Ap Loi Nhon, Tiền Giang, Vietnam', 'Ap My Tho, Tiền Giang, Vietnam', 'Ap Nhon My, An Giang, Vietnam',

In [396]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)

In [343]:
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G", "Field_3", "Field_38"],
    
    "cat_str" : ["Field_4", "Field_12","Field_36", "Field_54", "Field_62","Field_65","Field_66",\
                 "info_social_sex", "brief", "gioiTinh","currentLocationState", "Field_61", \
                 "currentLocationCountry", 'homeTownState', 'data.basic_info.locale',\
                "homeTownCountry", "Field_55"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh", "Field_34"],
    
    "address" : ["Field_45", "Field_48", "Field_49", "Field_56", 'homeTownName', "currentLocationCity", \
                 "homeTownCity", "diaChi", "currentLocationName"],
    
    "drop" : ["maCv", "Field_18", "Field_68", "Field_46"]
}

In [395]:
lst = data_type["cat_num"] + data_type["cat_str"] + data_type["date"] + data_type["address"] + data_type["drop"]

In [345]:
for f in df_transform.columns:
    vc = df_transform[f].value_counts()
    print(f"{f} : {len(vc)} : {vc.index[:10]}")

Field_1 : 28806 : Index(['2017-03-24T20:10:37.62Z', '2017-03-03T16:24:30.39Z',
       '2017-03-03T15:43:40.733Z', '2017-03-24T12:57:56.98Z',
       '2019-10-01T03:47:12Z', '2017-02-18T08:10:31.723Z',
       '2017-03-17T12:23:21.407Z', '2017-03-24T12:12:43.177Z',
       '2017-03-17T14:10:42.647Z', '2017-02-11T00:39:44.317Z'],
      dtype='object')
Field_2 : 24268 : Index(['2017-03-24T20:10:37.62Z', '2017-03-03T16:24:30.39Z',
       '2017-03-03T15:43:40.733Z', '2017-03-24T12:57:56.98Z',
       '2017-02-18T08:10:31.723Z', '2017-03-17T12:23:21.407Z',
       '2019-01-04T02:37:05Z', '2017-03-24T12:12:43.177Z',
       '2017-02-11T00:39:44.317Z', '2017-01-18T19:20:46.95Z'],
      dtype='object')
Field_3 : 2 : Float64Index([2.0, 1.0], dtype='float64')
Field_4 : 2 : Index(['T1', 'GH'], dtype='object')
Field_5 : 3101 : Index(['2018-12-27', '2018-12-28', '2018-12-26', '2018-12-25', '2018-12-29',
       '2018-12-24', '12/27/2018', '12/28/2018', '12/26/2018', '2019-01-02'],
      dtype='object')
Fie

Field_77 : 15 : Float64Index([4.0, 1.0, 3.0, 2.0, 5.0, 6.0, 7.0, 0.0, 8.0, 9.0], dtype='float64')
friendCount : 3411 : Float64Index([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0, 8.0], dtype='float64')
info_social_sex : 2 : Index(['male', 'female'], dtype='object')
subscriberCount : 1839 : Float64Index([0.0, 13.0, 47.0, 17.0, 21.0, 35.0, 26.0, 34.0, 12.0, 30.0], dtype='float64')
currentLocationLocationId : 1439 : Float64Index([              0.0, 108458769184495.0, 106388046062960.0,
              108459000000000.0, 106388000000000.0, 111711568847056.0,
              105667452800809.0, 114668461883395.0, 351759091676222.0,
              107983352568918.0],
             dtype='float64')
currentLocationLatitude : 772 : Float64Index([    0.0,   10.77, 21.0333, 16.0678, 19.8075, 20.8523, 10.9399,
              21.5839, 21.1723, 10.0359],
             dtype='float64')
currentLocationLongitude : 654 : Float64Index([     0.0, 106.6815,   105.85,  108.221,  105.776, 106.7005,
                105

consider transform: "brief" \
consider drop: "Field_56", "maCv"


In [346]:
df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["address"], axis = 1)

In [347]:
def fit38(x):
    if x == 0.0 or x == "0" or x == "0.0":
        return 0
    if x == 1.0 or x == "1":
        return 1
    return np.nan

In [348]:
def fit54(x):
    if x == "1" or x == "01" or x == ' P. Tân Phú':
        return "VN"
    if x in ['IT', 'GB', 'SE', 'TK', 'BD']:
        return np.nan
    return x

In [349]:
def fit55(x):
    if x in ['BC', 'BG', 'CN', 'CO', 'DB', 'EL', 'FO', 'H5', 'KX', 'N7', 'NR', 'SD', 'SJ', 'ST', 'WD']:
        return np.nan
    return x

In [350]:
def fit61(x):
    if x == 'Chất độc hóa học':
        return np.nan
    return x

In [351]:
def fitcurrentLocationState(x):
    if x in ['Ilhas', 'Daegu', 'Auckland Region', 'Basse-Normandie', 'Chongqing', "Provence-Alpes-Côte d'Azur",\
             'Rajshahi Division', 'Hubei', 'Mandalay', 'Missouri', 'Pathum Thani', 'Central Java', 'New Hampshire',\
             'Odisha', 'South Australia', 'Chon Buri', 'Haute-Normandie', 'Kirov Oblast', 'Primorsky Krai', 'Arizona',\
             'Ulsan', 'Kanagawa Prefecture', 'Metro Manila', 'Kachin State', 'Svay Rieng Province', 'Nagano Prefecture',\
             'Shiga Prefecture', 'Oslo', 'Indiana', 'Hunan', 'Bolikhamsai Province', 'Krasnoyarsk Krai',\
             'Distrito Nacional', 'Chiba Prefecture']:
        return np.nan
    return x

In [352]:
def fitcurrentLocationCountry(x):
    if x in ['Macau', 'New Zealand', 'Dominican Republic']:
        return np.nan
    return x

In [353]:
def fithomeTownState(x):
    if x in ['Hunan', 'Kachin State', 'Distrito Federal', 'Bizerte Governorate', 'Central Java',\
             'Svay Rieng Province', 'Distrito Nacional', 'County Mayo', 'Odisha', 'Lazio', 'Nevada',\
             'Daegu', 'Arizona', 'Equatoria', 'Fukuoka Prefecture', 'Taoyuan City', 'Kandal Province', \
             'State of Mexico', 'Hebei', 'New South Wales', 'Capital Governorate', 'Oregon']:
        return np.nan
    return x

In [354]:
def fithomeTownCountry(x):
    if x in ['Australia', 'Bahrain', 'Dominican Republic', 'Ireland', 'South Sudan', 'Tunisia']:
        return np.nan
    return x

In [355]:
def fitdata_basic_info_locale(x):
    if x == "_VN":
        return 'vi_VN'
    elif x in ['lv_LV', 'ru_RU']:
        return np.nan
    return x

In [383]:
def calculate_age(born):
    born = datetime.strptime(born,'%Y-%m-%d').date()
    today = datetime.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [357]:
def combine_gender(s):
    x, y = s
    
    if x != x and y != y:
        return "nan"
    
    if x != x:
        return y.lower()
    
    return x.lower()

def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1).astype("category")
    return df

In [376]:
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj
def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj

In [390]:
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        if col == "Field_34":
            continue
        df[col] = df[col].apply(date_normalize)

    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        df[col] = df[col].dt.strftime('%m-%Y')
    
    for cat in ['F', 'E', 'C', 'G', 'A']:
        df[f'{cat}_startDate'] = pd.to_datetime(df[f"{cat}_startDate"], infer_datetime_format=True)
        df[f'{cat}_endDate'] = pd.to_datetime(df[f"{cat}_endDate"], infer_datetime_format=True)
        
        df[f'{cat}_startDate'] = df[f'{cat}_startDate'].dt.strftime('%m-%Y')
        df[f'{cat}_endDate'] = df[f'{cat}_endDate'].dt.strftime('%m-%Y')
        
        cat_cols.append(f'{cat}_startDate')
        cat_cols.append(f'{cat}_endDate')
    
    for col in cat_cols:
        df[col] = df[col].astype("category")
        
    return df

In [380]:
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 34, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]

In [397]:
process_datetime_cols(df_transform)

AttributeError: Can only use .dt accessor with datetimelike values

In [359]:
df_transform["Field_38"] = df_transform["Field_38"].apply(fit38)
df_transform["Field_54"] = df_transform["Field_54"].apply(fit54)
df_transform["Field_55"] = df_transform["Field_55"].apply(fit55)
df_transform["Field_61"] = df_transform["Field_61"].apply(fit61)

df_transform["currentLocationState"] = df_transform["currentLocationState"].apply(fitcurrentLocationState)
df_transform["currentLocationCountry"] = df_transform["currentLocationCountry"].apply(fitcurrentLocationCountry)
df_transform["homeTownState"] = df_transform["homeTownState"].apply(fithomeTownState)
df_transform["homeTownCountry"] = df_transform["homeTownCountry"].apply(fithomeTownCountry)
df_transform['data.basic_info.locale'] = df_transform['data.basic_info.locale'].apply(fitdata_basic_info_locale)
df_transform["age"] = df_transform["ngaySinh"].map(lambda x: calculate_age(x) if x is not np.nan else x)

df_transform = process_gender(df_transform)
process_datetime_cols(df_transform)

In [364]:
drop = ["gioiTinh", "info_social_sex", "ngaySinh"]
df_transform = df_transform.drop(drop, axis = 1)

In [37]:
def transform_function(df):
    for feature in df.columns:
        if feature in data_type["cat_num"]:
            df[feature] = df[feature].fillna("Nan")
            #df = pd.get_dummies(df, columns=[feature])
        elif feature in data_type["date"]:
            df[feature] = df[feature].map(lambda x: float(parse(str(x)).timestamp()) if type(x) is str else x)
            df[feature].fillna(df[feature].mean())
        elif feature in  data_type["cat_str"]:
            df[feature] = df[feature].fillna("Nan")
            vc = df[feature].value_counts()
            df = pd.get_dummies(df, columns=[feature])
    return df

In [38]:
df_transform = transform_function(df_transform)
df_transform = df_transform.fillna(df_transform.mean())

In [71]:
scaler = MinMaxScaler()
scaler.fit(df_transform)
transform = scaler.transform(df_transform)
transform = pd.DataFrame(transform)
X_transfrom = transform.iloc[:len(df_train),:]
test_transform = transform.iloc[len(df_train):,:]

In [40]:
X1 = df_transform.Field_2 - df_transform.Field_1
X2 = df_transform.Field_6 - df_transform.Field_5
X3 = df_transform.Field_8 - df_transform.Field_7
X4 = df_transform.Field_33 - df_transform.Field_32
X5 = df_transform.Field_44 - df_transform.Field_43
X6 = df_transform.F_endDate - df_transform.F_startDate
X7 = df_transform.E_endDate - df_transform.E_startDate
X8 = df_transform.C_endDate - df_transform.C_startDate
X9 = df_transform.G_endDate - df_transform.G_startDate
X10 = df_transform.A_endDate - df_transform.A_startDate
df_transform = pd.concat([df_transform,X1,X2, X3, X4, X5, X6, X7, X8, X9, X10], axis=1)

In [93]:
train_dev = pd.concat([df_train.id, df_train.label,X_transfrom], axis = 1)
test_transform.reset_index(drop=True,inplace=True)
test = pd.concat([df_test.id, test_transform], axis = 1)

In [326]:
df_train["null_sum"] = df_train.isnull().sum(axis=1)

In [327]:
df_train["null_sum"]

0         46
1        111
2         34
3        141
4        141
        ... 
53025     20
53026     93
53027     31
53028    111
53029     85
Name: null_sum, Length: 53030, dtype: int64

In [96]:
params = {}
params['learning_rate'] = 0.01
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.1
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10

In [99]:
iter_predicttion = []
for i in range(4):
    pred_stack = []
    kf = KFold(n_splits = 8, shuffle=True)
    fold = kf.split(train_dev)
    for train_index, dev_index in fold:
        train1 = train_dev.iloc[train_index,1:]
        dev1 = train_dev.iloc[dev_index,1:]

        d_train = lgb.Dataset(train1, label = train_dev.loc[train_index]["label"])
        clf = lgb.train(params,d_train, 2500)
        predictions_test = clf.predict(test)
        pred_stack.append(predictions_test)
    predictions = np.asarray(pred_stack)
    predictions = np.mean(predictions, axis = 0)

    iter_predicttion.append(predictions)

In [100]:
predictions = np.asarray(iter_predicttion)
predictions = np.mean(predictions, axis = 0)
predictions = list(predictions)

for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
        
new_label = pd.DataFrame(predictions, columns = ["label"])

new_data_id = []
for i in range(len(new_label)):
    idx = len(df_train) + i
    new_data_id.append(idx)
new_data_id = pd.DataFrame(new_data_id, columns = ["id"])

new_data =  pd.concat([new_data_id, new_label, test.iloc[:,1:]], axis = 1)

,id,label,0,1,2,3,4,5,6,7,...,483,484,485,486,487,488,489,490,491,492
0,53030,1,0.930744,0.930721,0.967480,0.967480,0.891694,0.687620,0.967480,0.200000,...,1.067109e-07,0.539405,0.185971,0.394981,0.000000,0.048018,0.055092,0.037871,0.020911,0.126778
1,53031,1,0.901134,0.901937,0.971066,0.971066,0.889364,0.685944,0.971066,0.200000,...,9.970238e-04,0.539405,0.185971,0.477607,0.567141,0.048018,0.055092,0.037871,0.020911,0.000000
2,53032,1,0.704338,0.716350,0.881799,0.880360,0.806059,0.612649,0.882149,0.219541,...,1.451679e-02,0.535916,0.153460,0.465278,0.669977,0.048018,0.055092,0.037871,0.020911,0.126778
3,53033,1,0.936967,0.936973,0.981557,0.981557,0.903797,0.696328,0.981557,0.200000,...,3.542231e-05,0.539405,0.185971,0.590310,0.358795,0.048018,0.055092,0.037871,0.020911,0.126778
4,53034,1,0.704338,0.716350,0.881799,0.880360,0.806059,0.612649,0.882149,0.219541,...,1.451679e-02,0.535916,0.153460,0.465278,0.669977,0.000000,0.000000,0.000000,0.000000,0.117512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,73406,1,0.704338,0.716350,0.881799,0.880360,0.806059,0.612649,0.882149,0.219541,...,1.451679e-02,0.535916,0.153460,0.465278,0.669977,0.048018,0.055092,0.037871,0.020911,0.126778
20377,73407,1,0.704338,0.716350,0.881799,0.880360,0.806059,0.612649,0.882149,0.219541,...,1.451679e-02,0.535916,0.153460,0.465278,0.669977,0.048018,0.055092,0.037871,0.020911,0.126778
20378,73408,1,0.667007,0.667340,0.902919,0.902919,0.829723,0.704204,0.902919,0.200000,...,4.221206e-04,0.539405,0.334829,0.096539,0.796507,0.048018,0.000000,0.037871,0.020911,0.000000
20379,73409,1,0.704338,0.716350,0.881799,0.880360,0.806059,0.612649,0.882149,0.219541,...,1.451679e-02,0.535916,0.153460,0.465278,0.669977,0.000000,0.000000,0.037871,0.000000,0.002304


In [101]:
data_version = "07-10-2020-full"

In [103]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train_dev.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)
new_data.to_csv(f"../../data/kalapa/{data_version}/new_data.csv", index = False)